IMPORTING THE FILES:

In [53]:
import pandas as pd

DATA_PATH = '../data'

TASK 1:
Your boss wants to know how electricity prices in NO2 and Germany have developed over the 
sample period. Therefore, she has asked you to create a line plot that visualizes the hourly 
electricity price in NO2 and Germany. The graph should: 
- consist of two subplots: 
o The first subplot shows the hourly electricity price in NO2 over time. 
o The second subplot shows the hourly electricity price in Germany over time. 
- add a vertical line to each subplot to indicate the official opening of Nordlink (December 
9, 2020).  
- be stored as a png file called “figure_task1.png”. 

What is your conclusion? How has the electricity price in NO2 and Germany developed over 
time? Have the prices increased or decreased after the opening of Nordlink? Extra: Do you have 
any suggestions for why we sometimes observe negative electricity prices?

TASK 2:
Your boss has heard that electricity prices have become more volatile with extreme prices in 
recent years. Therefore, she has asked you to create a table with some descriptive statistics for 
the hourly electricity price in NO2 and Germany. The table should: 
- contain the mean, median, standard deviation, min and max of the hourly electricity 
price in NO2 and Germany separately for each year in the sample (2019, 2020, 2021, 
2022, 2023). 
- round all descriptive statistics to two decimals. 
- be stored as an excel file called “table_task2.xlsx”. 
What is your conclusion? Have electricity prices in NO2 and Germany become more extreme in 
recent years? Have the prices evolved differently in NO2 than in Germany?

TASK 3:
Your boss is interested in knowing whether NO2 exports more electricity than it imports from 
Germany, and if this has changed over time. Therefore, she has asked you to create a bar plot 
that shows the annual sums of imports and exports of electricity in NO2. The graph should: 
- contain a single bar plot that shows the annual sums of exports and imports of electricity 
in NO2 from Nordlink for each year in the data. 
- be stored as a png file called “figure_task3.png”. 
What is your conclusion? Does NO2 import or export more electricity to Germany? Have the 
annual sums of exports and imports changed over the years? 

TASK 4:
Your boss wants to take a closer look at the transmission flows and investigate whether there 
are times during the year in which NO2 imports more electricity from Germany than it exports. 
Specifically, she wants to know whether there have been weeks in the data in which the sum of 
net exports of electricity from NO2 to Germany has been negative (i.e., NO2 imports more 
electricity than it exports to Germany). Note that the net export is defined as the export of 
electricity minus the import of electricity.  
Therefore, she has asked you to create a line plot that shows the weekly sum of net exports of 
electricity from NO2 to Germany. The graph should: 
- contain a single line plot that shows the weekly sum of net exports of electricity from 
NO2 to Germany over time. 
- contain a horizontal line at y=0 to indicate when the weekly net export is zero. 
- be stored as a png file called “figure_task4.png”. 
What is your conclusion? Have NO2 always exported more electricity to Germany than it has 
imported? Or have there been weeks in the data in which NO2 was a net importer of electricity? 
If so, at what times during the year does this usually occur?

TASK 5: 
Your boss believes that NO2 tends to export more electricity to Germany whenever the 
electricity price in Germany is higher relative to the price in NO2. She has asked you to 
investigate this claim by creating a scatter plot between the weekly sum of net exports in NO2 
and the weekly average price difference between Germany and NO2. Note that to find the 
weekly average price difference, you should first find the hourly price difference, and then take 
the weekly average. 
The graph should: 
- contain a single scatter plot that shows the weekly sum of net exports in NO2 on the y-
axis and the weekly average price difference between Germany and NO2 on the x-axis. 
- contain the correlation coefficient between the weekly sum of net exports and the 
weekly average price difference in the figure title. 
- be stored as a png file called “figure_task5.png”. 
What is your conclusion? Does there seem to be a relationship between the weekly sum of net 
exports and the weekly average price difference? If so, is the relationship positive or negative? 

TASK 6:
Your boss is curious to know how you have used online sources in completing the assignment. 
She has asked you to write a statement describing your use of online sources, e.g., 
Stackoverflow, official function documentation, generative AI etc. If you have copied code from 
online, you can use this statement to point out the code snippets that you have not written 
yourself along with a reference to the source. In addition, if you have used generative AI, e.g., 
ChatGPT, your statement should explain how you have used such tools to help with the 
assignment.